<a href="https://colab.research.google.com/github/TokusimaHonda/DataScience/blob/main/GetYahooNewsTopics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse

In [ ]:
def get_yahoo_news_topics_list(category_url):
    response = requests.get(category_url)
    soup = BeautifulSoup(response.text, "html.parser")
    #主要のピックアップを取得
    topics_tags = soup.find_all(href=re.compile("news.yahoo.co.jp/pickup"))
    #ループ
    topics_list = []
    for topics_tag in topics_tags:
        topics_dic = {
            'url':topics_tag.attrs['href'],
            'title':topics_tag.text,
        }
        topics_list.append(topics_dic)
    return topics_list

In [ ]:
get_yahoo_news_topics_list('https://news.yahoo.co.jp')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/domestic')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/world')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/business')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/entertainment')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/sports')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/it')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/science')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/life')
get_yahoo_news_topics_list('https://news.yahoo.co.jp/categories/local')

[{'title': '調布の陥没 1000軒に補償検討',
  'url': 'https://news.yahoo.co.jp/pickup/6384925'},
 {'title': '都知事 五輪ボラ戻ってきて', 'url': 'https://news.yahoo.co.jp/pickup/6384900'},
 {'title': '26年前の震災 死を覚悟した3h',
  'url': 'https://news.yahoo.co.jp/pickup/6384858'},
 {'title': '複数箇所刺され女性死亡 熊本', 'url': 'https://news.yahoo.co.jp/pickup/6384909'},
 {'title': 'イノシシに襲われ 79歳死亡', 'url': 'https://news.yahoo.co.jp/pickup/6384899'},
 {'title': '福岡少年院 一律丸刈りを変更', 'url': 'https://news.yahoo.co.jp/pickup/6384861'},
 {'title': '0歳死亡ベッドガード巡り提訴', 'url': 'https://news.yahoo.co.jp/pickup/6384894'},
 {'title': '温泉でキャビア 40年苦労実る',
  'url': 'https://news.yahoo.co.jp/pickup/6384866'},
 {'title': '調布の陥没 1000軒に補償検討',
  'url': 'https://news.yahoo.co.jp/pickup/6384925'}]

In [ ]:
#ニュースタイトル+時間含めて取得
test = get_yahoo_news_topics_list('https://news.yahoo.co.jp/topics/top-picks')
test[1]

{'title': '五輪ボラ辞退 740人に拡大2/12(金) 21:30',
 'url': 'https://news.yahoo.co.jp/pickup/6384917'}

In [ ]:
def url_add_query(url, key,new_val):
    pr = urllib.parse.urlparse(url)
    d = urllib.parse.parse_qs(pr.query)
    d[key] = new_val
    return urllib.parse.urlunparse(pr._replace(query=urllib.parse.urlencode(d, doseq=True)))

In [ ]:
#https://news.yahoo.co.jp/topics/top-picks
#https://news.yahoo.co.jp/topics/top-picks?date=20210209
def get_category_page_url(category_url):
    response = requests.get(category_url)
    soup = BeautifulSoup(response.text, "html.parser")
    pagination_number_current = soup.find(class_=re.compile("pagination_number_current")).text
    pagination_number = soup.find(class_=re.compile("pagination_number_text")).text
    page_topics_max=re.findall('〜(.*)件', pagination_number_current)[0]
    topics_count=pagination_number.replace("件","")
    #切り上げ除算をしてページ数を算出
    page_count = -(-(int(topics_count)) // int(page_topics_max))
    url_list = []
    for page_num in range(1,page_count+1):
        page_url = url_add_query(category_url, 'page', page_num)
        url_list.append(page_url)
    
    return url_list


In [ ]:
get_category_page_url("https://news.yahoo.co.jp/topics/top-picks?date=20210209")

['https://news.yahoo.co.jp/topics/top-picks?date=20210209&page=1',
 'https://news.yahoo.co.jp/topics/top-picks?date=20210209&page=2',
 'https://news.yahoo.co.jp/topics/top-picks?date=20210209&page=3',
 'https://news.yahoo.co.jp/topics/top-picks?date=20210209&page=4']

In [ ]:
#https://news.yahoo.co.jp/topics/top-picks
#https://news.yahoo.co.jp/topics/top-picks?date=20210209
def get_yahoo_news_category_topics_all_list(category_url):
    page_url_list = get_category_page_url(category_url)
    topics_all_list = []
    for page_url in page_url_list:
        response = requests.get(page_url)
        soup = BeautifulSoup(response.text, "html.parser")
        #ピックアップを取得
        topics_tags = soup.find_all(href=re.compile("news.yahoo.co.jp/pickup"))
        for topics_tag in topics_tags:
            text_list = list(topics_tag.strings)
            if len(text_list) == 2:
                topics_dic = {
                    'topic_url':topics_tag.attrs['href'],
                    'article_url':pickup_url_to_article_url(topics_tag.attrs['href']),
                    'title':text_list[0],
                    'date':text_list[1],
                }
                topics_all_list.append(topics_dic)
            elif len(text_list) == 3:
                topics_dic = {
                    'topic_url':topics_tag.attrs['href'],
                    'article_url':pickup_url_to_article_url(topics_tag.attrs['href']),
                    'title':text_list[1],
                    'date':text_list[2],
                }
                topics_all_list.append(topics_dic)
    return topics_all_list

In [ ]:
get_yahoo_news_category_topics_all_list("https://news.yahoo.co.jp/topics/top-picks")

[{'article_url': 'https://headlines.yahoo.co.jp/hl?a=20210213-00000022-mai-n_ame',
  'date': '2/13(土) 12:12',
  'title': 'トランプ氏「無罪」の公算大',
  'topic_url': 'https://news.yahoo.co.jp/pickup/6384957'},
 {'article_url': 'https://headlines.yahoo.co.jp/hl?a=20210213-00010001-nishinpc-pol',
  'date': '2/13(土) 12:08',
  'title': '医療界の反感懸念、宣言継続',
  'topic_url': 'https://news.yahoo.co.jp/pickup/6384958'},
 {'article_url': 'https://headlines.yahoo.co.jp/hl?a=20210213-00010000-nikkei-bus_all',
  'date': '2/13(土) 11:49',
  'title': '忍び寄る中国 サムスンの暗闘',
  'topic_url': 'https://news.yahoo.co.jp/pickup/6384956'},
 {'article_url': 'https://headlines.yahoo.co.jp/hl?a=20210213-00050043-yom-soci',
  'date': '2/13(土) 11:30',
  'title': '天皇ご一家の荷物 皇居へ搬送',
  'topic_url': 'https://news.yahoo.co.jp/pickup/6384955'},
 {'article_url': 'https://news.yahoo.co.jp/byline/kawamurameikou/20210213-00222230/',
  'date': '2/13(土) 11:15',
  'title': 'ゲーム障害巡る投稿 問題点は',
  'topic_url': 'https://news.yahoo.co.jp/pickup/6384953'},
 {

In [ ]:
def pickup_url_to_article_url(pickup_url):
    response = requests.get(pickup_url)
    soup = BeautifulSoup(response.text, "html.parser")
    article_tag = soup.find('a' ,text=re.compile('続きを読む'))
    if article_tag is not None:
        article_url = article_tag.attrs['href']
    else:
        sport_article_tag = soup.find('a' ,text=re.compile('速報を見る'))
        if sport_article_tag is not None:
            article_url = sport_article_tag.attrs['href']
        else:
            article_url = None
    return article_url

In [ ]:
pickup_url_to_article_url("https://news.yahoo.co.jp/pickup/6384952")

'https://headlines.yahoo.co.jp/hl?a=20210213-01049452-fullcount-base'